# The Tensorflow Way

How to put together objects using eager to dynamically setup the computational graph. Eventually create a classifier.

## Outcomes

* Operations using eager execution
* Layering nested operations
* Working with multiple layers
* Implementing loss functions
* Implementing backpropagation
* Working with batch and stochastic training
* Combining components together

| Date | User | Change Type | Remarks |  
| ---- | ---- | ----------- | ------- |
| 14/10/2024   | Martin | Created   | Started chapter 2 | 

# Content

* [Operations using eager execution](#header-1)

# Operations using eager execution

Operating on matrices using eager execution. 

Eager excution allows you to perform operations directly on the results, instead of working on symbolic handles of a computational graph.

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["GRPC_VERBOSITY"] = "ERROR"
os.environ["GLOG_minloglevel"] = "2"

import tensorflow as tf
import numpy as np

In [2]:
# Convert a numpy array into a tensor and perform an operation
x_vals = np.array([1., 3., 5., 7., 9.])
x_data = tf.Variable(x_vals, dtype=tf.float32)
x_const = tf.constant(3.)

operation = tf.multiply(x_data, x_const)
for val in operation: # where eager execution happens
  print(val.numpy())

3.0
9.0
15.0
21.0
27.0


# Layering nested operations

Putting multiple operations together. Multiply 2 matrics then perform addition, each matrix is 3D.

Use common constructs like functions and classes to improve readability and code modularity.

In [9]:
# Starting matrix
array = np.array([[1., 3., 5., 7., 9.],
                  [-2., 0., 2., 4., 6.],
                  [-6., -3., 0., 3., 6.]]) # 3x5
x_vals = np.array([array, array + 1]) # 3x5x2
x_data = tf.Variable(x_vals, dtype=tf.float32) # 3x5x2

# Operation matrices
m1 = tf.constant([[1.], [0.], [-1.], [2.], [4.]]) # 5x1
m2 = tf.constant([[2.]]) # 1x1
a1 = tf.constant([[10.], [3.], [1.]]) # 3x1

def prod(a, b):
  return tf.matmul(a, b)
def add(a, b):
  return tf.add(a, b)

# Operations
result = add(prod(prod(x_data, m1), m2), a1)
print(result.numpy())

[[[102.]
  [ 59.]
  [ 49.]]

 [[114.]
  [ 71.]
  [ 61.]]]


Prevent "kichen sink" programming style (putting everything in the global scope of the program). Adopt a functional or object-oriented programming style

In [11]:
class Operations():
  def __init__(self, a):
    self.result = a
  def apply(self, func, b):
    self.result = func(self.result, b)
    return self

operation = (
  Operations(a=x_data)
  .apply(prod, b=m1)
  .apply(prod, b=m2)
  .apply(add, b=a1)
)
print(operation.result.numpy())

[[[102.]
  [ 59.]
  [ 49.]]

 [[114.]
  [ 71.]
  [ 61.]]]


If the shape of the tensor is not known beforehand, we can initialise the unknown dimension with `None`.

In [15]:
# Initialise with unknown dimension
v = tf.Variable(
  initial_value=tf.random.normal(shape=(1, 5)),
  shape=tf.TensorShape((None, 5))
)
v.assign(tf.random.normal(shape=(10, 5)))

<tf.Variable 'UnreadVariable' shape=(None, 5) dtype=float32, numpy=
array([[-0.2408374 ,  1.7669061 , -0.9611152 ,  1.7895854 , -0.6851607 ],
       [ 1.9162391 ,  1.0384045 , -0.8034953 ,  0.33766457, -0.54708785],
       [-3.0316987 , -2.0763571 ,  1.5104392 ,  0.3365694 ,  0.74961025],
       [ 1.4060197 , -0.91576695, -0.60831064,  0.45539996, -0.8254257 ],
       [-1.1942742 , -0.31644046, -0.85949975, -1.147775  , -0.57103795],
       [ 1.234363  ,  1.9150587 ,  0.33312622,  0.4733555 ,  1.5149691 ],
       [ 1.1402022 , -1.8850285 , -0.11132842, -1.9180793 ,  1.7446767 ],
       [-0.48002127, -0.2189978 ,  0.6621816 ,  1.2813387 ,  0.7167656 ],
       [ 1.226478  ,  1.2447158 ,  1.0185527 ,  1.6968899 ,  0.64589614],
       [-0.37618685,  2.0898812 , -0.2646985 , -0.37295765,  0.9064721 ]],
      dtype=float32)>

# Working with multiple layers

How to connect data layers together. Example will be image processing by (1) average through a moving window (2) custom operation layer.

As more layers are added, the computational graph can get complicated. Therefore we introduce scopes to group the layers together

In [8]:
# Create an "image" 4x4 pixel image
batch_size = [1]
x_shape = [4, 4, 1]
x_data = tf.random.uniform(shape=batch_size + x_shape)

In [9]:
# Conv2D - moving window
def mov_avg_layer(x):
  """
  Formula takes the average of all values in the window
  """
  my_filter = tf.constant(0.25, shape=[2, 2, 1, 1])
  my_strides = [1, 2, 2, 1]
  layer = tf.nn.conv2d(
    x,
    my_filter,
    my_strides,
    padding='SAME',
    name='Moving_Avg_Window'
  )
  return layer
mov_avg_layer(x_data)

<tf.Tensor: shape=(1, 2, 2, 1), dtype=float32, numpy=
array([[[[0.48430553],
         [0.5037121 ]],

        [[0.66042995],
         [0.77765524]]]], dtype=float32)>

💡 Note that the formula for calculating the output is the following:

$$
Output = \frac{(W-F+2P)}{S+1}\\
W:\ input\ size\\
F:\ filter\ size\\
P:\ padding\\
S:\ stride
$$

In [10]:
def custom_layer(input_matrix):
  # remove unecessary dimension
  reduce_dimensions = tf.squeeze(input_matrix)
  
  # define matrices 
  A = tf.constant([[1., 2.], [-1., 3.]])
  b = tf.constant(1., shape=[2, 2])

  # compute Ax + b
  output = tf.matmul(A, reduce_dimensions)
  output = tf.add(output, b)

  return output

In [11]:
# Implement the layers
first_layer = mov_avg_layer(x_data)
second_layer = custom_layer(first_layer)

print(second_layer)

tf.Tensor(
[[2.8051653 3.0590227]
 [2.4969845 2.8292537]], shape=(2, 2), dtype=float32)
